# Import All Libaries

In [2]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
ensure_pyterrier_is_loaded()
import pandas as pd
import pyterrier as pt
from tqdm import tqdm
from jnius import autoclass
import gzip
import json
import math

# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
tira = Client()


import nltk
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Load the Dataset and the Index

In [3]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
dataset = 'ir-acl-anthology-20240504-training'
pt_dataset = pt.get_dataset(f'irds:ir-lab-sose-2024/{dataset}')


# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

### Stemmer and Lemmatizer  (funktioniert leider nicht)

Variante 1

In [4]:
'''def stem(t):
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(t)

lemmatizer = WordNetLemmatizer()


indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, stemmer=lemmatizer)
index_ref = indexer.index(pt_dataset.get_corpus_iter())
index = pt.IndexFactory.of(index_ref)

bm25_stem = pt.BatchRetrieve(index, wmodel="BM25")



def stem(t):
    stemmer = LancasterStemmer()
    return stemmer.stem(t)

stemmer = LancasterStemmer()

indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, stemmer=stemmer)
index_ref = indexer.index(pt_dataset.get_corpus_iter())
index = pt.IndexFactory.of(index_ref)


bm25_lem = pt.BatchRetrieve(index, wmodel="BM25")'''





'def stem(t):\n    lemmatizer = WordNetLemmatizer()\n    return lemmatizer.lemmatize(t)\n\nlemmatizer = WordNetLemmatizer()\n\n\nindexer = pt.IterDictIndexer("/tmp/index", overwrite=True, stemmer=lemmatizer)\nindex_ref = indexer.index(pt_dataset.get_corpus_iter())\nindex = pt.IndexFactory.of(index_ref)\n\nbm25_stem = pt.BatchRetrieve(index, wmodel="BM25")\n\n\n\ndef stem(t):\n    stemmer = LancasterStemmer()\n    return stemmer.stem(t)\n\nstemmer = LancasterStemmer()\n\nindexer = pt.IterDictIndexer("/tmp/index", overwrite=True, stemmer=stemmer)\nindex_ref = indexer.index(pt_dataset.get_corpus_iter())\nindex = pt.IndexFactory.of(index_ref)\n\n\nbm25_lem = pt.BatchRetrieve(index, wmodel="BM25")'

Variante 2

In [5]:
'''def stem_text(text):
    stemmer = LancasterStemmer()
    return " ".join([stemmer.stem(word) for word in nltk.word_tokenize(text)])

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text)])

def document_iterator(docs, transform_func):
    for doc in docs:
        doc["text"] = transform_func(doc["text"])
        yield doc


transform_func = stem_text  # oder lemmatize_text


indexer = pt.IterDictIndexer("/tmp/index", overwrite=True)
index_ref = indexer.index(document_iterator(pt_dataset.get_corpus_iter(), transform_func))

class NLTKStemmerTransformer(pt.Transformer):
    def __init__(self, stemmer):
        self.stemmer = stemmer

    def transform(self, queries):
        queries['query'] = queries['query'].apply(stem_text)
        return queries


stemmer = LancasterStemmer()
nltk_stemmer_transformer = NLTKStemmerTransformer(stemmer)


index = pt.IndexFactory.of(index_ref)


bm25_stem = nltk_stemmer_transformer >> pt.BatchRetrieve(index, wmodel="BM25")'''

'def stem_text(text):\n    stemmer = LancasterStemmer()\n    return " ".join([stemmer.stem(word) for word in nltk.word_tokenize(text)])\n\ndef lemmatize_text(text):\n    lemmatizer = WordNetLemmatizer()\n    return " ".join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text)])\n\ndef document_iterator(docs, transform_func):\n    for doc in docs:\n        doc["text"] = transform_func(doc["text"])\n        yield doc\n\n\ntransform_func = stem_text  # oder lemmatize_text\n\n\nindexer = pt.IterDictIndexer("/tmp/index", overwrite=True)\nindex_ref = indexer.index(document_iterator(pt_dataset.get_corpus_iter(), transform_func))\n\nclass NLTKStemmerTransformer(pt.Transformer):\n    def __init__(self, stemmer):\n        self.stemmer = stemmer\n\n    def transform(self, queries):\n        queries[\'query\'] = queries[\'query\'].apply(stem_text)\n        return queries\n\n\nstemmer = LancasterStemmer()\nnltk_stemmer_transformer = NLTKStemmerTransformer(stemmer)\n\n\nindex = pt.IndexF

In [6]:
'''
pt.Experiment(
    retr_systems=[bm25, bm25_stem],
    topics=pt_dataset.get_topics('text'),
    qrels=pt_dataset.get_qrels(),
    names=['BM25', 'bm25_stem'],
    eval_metrics=['recall_1000', 'ndcg_cut_5', 'ndcg_cut.10', 'recip_rank']
)'''



"\npt.Experiment(\n    retr_systems=[bm25, bm25_stem],\n    topics=pt_dataset.get_topics('text'),\n    qrels=pt_dataset.get_qrels(),\n    names=['BM25', 'bm25_stem'],\n    eval_metrics=['recall_1000', 'ndcg_cut_5', 'ndcg_cut.10', 'recip_rank']\n)"

Leider kommt immer wieder 0.0 raus egal welche Variante ich benutze.

## Document Expansion by Query Prediction with docT5query
The basic idea is to train a model, that when given an input document, generates questions that the document might answer (or more broadly, queries for which the document might be relevant). These predicted questions (or queries) are then appended to the original documents, which are then indexed as before. The docT5query model gets its name from the use of T5 as the expansion model.

The primary advantage of this approach is that expensive neural inference is pushed to indexing time, which means that "bag of words" queries against an inverted index built on the augmented document collection are only slightly slower (due to longer documents) — but the retrieval results are much better.

First we check, if our corpus has a high recall or a lower. Our Corpus in this case is the union of the IR Anthology and the ACL Anthology. The recall may change if we use another corpus.

In [7]:
'''bm25 = pt.BatchRetrieve(index, wmodel="BM25")

pt.Experiment(
    retr_systems=[bm25],
    topics=pt_dataset.get_topics('text'),
    qrels=pt_dataset.get_qrels(),
    names=['BM25'],
    eval_metrics=['recall_1000']
)'''

'bm25 = pt.BatchRetrieve(index, wmodel="BM25")\n\npt.Experiment(\n    retr_systems=[bm25],\n    topics=pt_dataset.get_topics(\'text\'),\n    qrels=pt_dataset.get_qrels(),\n    names=[\'BM25\'],\n    eval_metrics=[\'recall_1000\']\n)'

As we can see, we have already a high recall. This is important for the way we implement the docT5query.
More information about the implementation in the [Tutorial](https://github.com/tira-io/teaching-ir-with-shared-tasks/blob/main/tutorials/tutorial-doc-t5-query.ipynb).

In [8]:
'''def doc_t5_query(dataset):
    docs = tira.get_run_output('ir-benchmarks/seanmacavaney/DocT5Query', dataset) + '/documents.jsonl.gz'
    with gzip.open(docs, 'rt') as f:
        for l in tqdm(f):
            l = json.loads(l)
            l['text'] = l['querygen']
            l['docno'] = l['doc_id']
            del l['doc_id']
            del l['querygen']
            yield l

def doc_t5_query_index(dataset):
    indexer = pt.IterDictIndexer("/tmp/index2", overwrite=True, meta={'docno': 100, 'text': 20480})
    index_ref = indexer.index(doc_t5_query(dataset))
    return pt.IndexFactory.of(index_ref)'''


'def doc_t5_query(dataset):\n    docs = tira.get_run_output(\'ir-benchmarks/seanmacavaney/DocT5Query\', dataset) + \'/documents.jsonl.gz\'\n    with gzip.open(docs, \'rt\') as f:\n        for l in tqdm(f):\n            l = json.loads(l)\n            l[\'text\'] = l[\'querygen\']\n            l[\'docno\'] = l[\'doc_id\']\n            del l[\'doc_id\']\n            del l[\'querygen\']\n            yield l\n\ndef doc_t5_query_index(dataset):\n    indexer = pt.IterDictIndexer("/tmp/index2", overwrite=True, meta={\'docno\': 100, \'text\': 20480})\n    index_ref = indexer.index(doc_t5_query(dataset))\n    return pt.IndexFactory.of(index_ref)'

In [9]:
#indexD = doc_t5_query_index(dataset)

In [10]:
'''docs_retrieved_by_bm25 = {}

bm25_result = bm25(pt_dataset.get_topics('title'))

for _, i in tqdm(bm25_result.iterrows()):
    qid, docno = str(i['qid']), str(i['docno'])

    if qid not in docs_retrieved_by_bm25:
        docs_retrieved_by_bm25[qid] = set()
    
    docs_retrieved_by_bm25[qid].add(docno)'''

"docs_retrieved_by_bm25 = {}\n\nbm25_result = bm25(pt_dataset.get_topics('title'))\n\nfor _, i in tqdm(bm25_result.iterrows()):\n    qid, docno = str(i['qid']), str(i['docno'])\n\n    if qid not in docs_retrieved_by_bm25:\n        docs_retrieved_by_bm25[qid] = set()\n    \n    docs_retrieved_by_bm25[qid].add(docno)"

In [11]:
'''omit_already_retrieved_docs = lambda i: i[i.apply(lambda j: str(j['docno']) not in docs_retrieved_by_bm25[str(j['qid'])], axis=1)]
omit_already_retrieved_docs = pt.apply.generic(omit_already_retrieved_docs)'''

"omit_already_retrieved_docs = lambda i: i[i.apply(lambda j: str(j['docno']) not in docs_retrieved_by_bm25[str(j['qid'])], axis=1)]\nomit_already_retrieved_docs = pt.apply.generic(omit_already_retrieved_docs)"

In [12]:
'''bm25_doct5query = pt.BatchRetrieve(indexD, wmodel="BM25")
bm25_doct5query_new = bm25_doct5query >> omit_already_retrieved_docs'''

'bm25_doct5query = pt.BatchRetrieve(indexD, wmodel="BM25")\nbm25_doct5query_new = bm25_doct5query >> omit_already_retrieved_docs'

DocT5Query bringt uns im normalen dataset nicht so viel, also lassen wir es weg

## Stopwords

In [33]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

def create_index(documents, stopwords):
    indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stopwords=customStopwords)
    index_ref = indexer.index(documents)
    return pt.IndexFactory.of(index_ref)

customStopwords =[
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 
    'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 
    'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 
    'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 
    'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 
    'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 
    'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 
    'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 
    'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 
    'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 
    'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 
    'will', 'just', 'don', 'should', 'now', 'e', 'n', 'd', 'o', 'c', 'r'
]

index = create_index(pt_dataset.get_corpus_iter(), customStopwords)

bm25_stopwords = pt.BatchRetrieve(index, wmodel="BM25")

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  71%|███████   | 90198/126958 [00:18<00:06, 6118.14it/s]

15:38:26.135 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:24<00:00, 5238.78it/s] 


15:38:34.169 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


## Query Expansion with Large Language Models

In [34]:

# Ensure BM25 is initialized
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
bm25_rm3 = bm25 >> pt.rewrite.RM3(index) >> bm25
bm25_kl = bm25 >> pt.rewrite.KLQueryExpansion(index) >> bm25

# llm expansions with gpt
gpt_cot = tira.pt.transform_queries('workshop-on-open-web-search/tu-dresden-03/qe-gpt3.5-cot', dataset, prefix='llm_expansion_')
gpt_sq_fs = tira.pt.transform_queries('workshop-on-open-web-search/tu-dresden-03/qe-gpt3.5-sq-fs', dataset, prefix='llm_expansion_')
gpt_sq_zs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-gpt3.5-sq-zs', dataset, prefix='llm_expansion_')

# llm expansions with llama
llama_cot = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-llama-cot', dataset, prefix='llm_expansion_')
llama_sq_fs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-llama-sq-fs', dataset, prefix='llm_expansion_')
llama_sq_zs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-llama-sq-zs', dataset, prefix='llm_expansion_')



In [57]:
tokeniser = pt.autoclass("org.terrier.indexing.tokenisation.Tokeniser").getTokeniser()

documents = []
for doc in pt_dataset.get_corpus_iter():
        documents.append({
        'docno': doc['docno'],
        'text': doc['text'],
})   

def pt_tokenize(text):
    return ' '.join(tokeniser.getTokens(text))

# we wrap this into an pyterrier transformer
# Documentation: https://pyterrier.readthedocs.io/en/latest/apply.html

def calculate_tf_idf(term, doc_text, index):
    # Implement your TF-IDF calculation logic here
    # Example placeholder implementation:
    tf = doc_text.lower().count(term.lower()) / len(doc_text.split())
    #print(tf)
    num_docs = index.getCollectionStatistics().getNumberOfDocuments()
    lexicon = index.getLexicon()
    entry = lexicon.getLexiconEntry(term.lower())
    doc_freq = entry.getDocumentFrequency() if entry is not None else 0
    idf = math.log((num_docs + 1) / (doc_freq + 1)) + 1

    #print(tf * idf)
    return tf * idf

# Define a function to retrieve top documents and extract terms with scores
def retrieve_top_docs_terms(query, num_docs=10, num_terms=5):
    # Annahme: bm25 ist ein BatchRetrieve-Objekt
    from jnius import autoclass
    tokeniser = autoclass("org.terrier.indexing.tokenisation.Tokeniser").getTokeniser()
    query = " ".join(tokeniser.getTokens(query))
    query_results = bm25.search(query)
    top_docs = query_results.head(num_docs)['docid'].tolist()
    terms_scores = {}
    
    for docid in top_docs:
        
        doc_text = documents[docid]['text']
        doc_terms = pt_tokenize(doc_text).split()
        doc_score = query_results[query_results['docid'] == docid]['score'].values[0]
        for x in range(10):
            for term in doc_terms:
                if term in customStopwords:
                    doc_terms.remove(term)
            x += 1

        for term in doc_terms:
            if term not in terms_scores:
                terms_scores[term] = 0
            terms_scores[term] += doc_score * calculate_tf_idf(term, doc_text, index)
    
    sorted_terms = sorted(terms_scores.items(), key=lambda x: x[1], reverse=True)
   
    
    return [term for term, score in sorted_terms[:num_terms]]
        


# Function to expand query
def expand_query(topic):
    original_query = topic['query']
    #expanded_terms = retrieve_top_docs_terms(original_query)
    llm_query = topic['llm_expansion_query']
    #print("Test")
   
    
    #print(llm_query)
    
    llm_terms = retrieve_top_docs_terms(llm_query)
    #print(llm_terms)
    
    expanded_query = ' '.join([original_query] * 6 + llm_terms)

    return pt_tokenize(expanded_query)

# Wrapper for PyTerrier query expansion
pt_expand_query = pt.apply.query(expand_query)


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:02<00:00, 56287.99it/s]


In [60]:
# Update pipelines with the new query expansion function
pipeline_gpt_cot = (gpt_cot >> pt_expand_query) >> bm25
pipeline_gpt_sq_fs = (gpt_sq_fs >> pt_expand_query) >> bm25
pipeline_gpt_sq_zs = (gpt_sq_zs >> pt_expand_query) >> bm25

pipeline_llama_cot = (llama_cot >> pt_expand_query) >> bm25
pipeline_llama_sq_fs = (llama_sq_fs >> pt_expand_query) >> bm25
pipeline_llama_sq_zs = (llama_sq_zs >> pt_expand_query) >> bm25

In [61]:
pt.Experiment(
    retr_systems=[bm25, bm25_kl, pipeline_gpt_cot, pipeline_gpt_sq_fs, pipeline_gpt_sq_zs, pipeline_llama_cot, pipeline_llama_sq_fs, pipeline_llama_sq_zs],
    topics=pt_dataset.get_topics(),
    qrels=pt_dataset.get_qrels(),
    names=['BM25','BM25+KL', 'BM25+GPT-COT', 'BM25+GPT-SQ-FS', 'BM25+GPT-SQ-ZS', 'BM25+Llama-COT', 'BM25+Llama-SQ-FS', 'BM25+Llama-SQ-ZS'],
    eval_metrics=['recall_1000', 'ndcg_cut_5', 'ndcg_cut_10', 'recip_rank']
)
 


There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


,name,recall_1000,ndcg_cut_5,ndcg_cut_10,recip_rank
0,BM25,0.834068,0.375888,0.363774,0.572496
1,BM25+KL,0.839492,0.379332,0.362127,0.598772
2,BM25+GPT-COT,0.847365,0.403628,0.381031,0.604876
3,BM25+GPT-SQ-FS,0.849050,0.358546,0.346663,0.569714
4,BM25+GPT-SQ-ZS,0.845811,0.370241,0.352965,0.574072
5,BM25+Llama-COT,0.839724,0.364432,0.355543,0.568273
6,BM25+Llama-SQ-FS,0.847496,0.390159,0.362397,0.599283
7,BM25+Llama-SQ-ZS,0.847622,0.395643,0.372658,0.619827


Ab hier gibt es nur noch alte Versuche

In [19]:
'''bm25_rm3 = bm25 >> pt.rewrite.RM3(index) >> bm25
bm25_kl = bm25 >> pt.rewrite.KLQueryExpansion(index) >> bm25'''

'bm25_rm3 = bm25 >> pt.rewrite.RM3(index) >> bm25\nbm25_kl = bm25 >> pt.rewrite.KLQueryExpansion(index) >> bm25'

In [20]:
'''# llm expansions with gpt
gpt_cot = tira.pt.transform_queries('workshop-on-open-web-search/tu-dresden-03/qe-gpt3.5-cot', dataset, prefix='llm_expansion_')
gpt_sq_fs = tira.pt.transform_queries('workshop-on-open-web-search/tu-dresden-03/qe-gpt3.5-sq-fs', dataset, prefix='llm_expansion_')
gpt_sq_zs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-gpt3.5-sq-zs', dataset, prefix='llm_expansion_')

# llm expansions with llama
llama_cot = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-llama-cot', dataset, prefix='llm_expansion_')
llama_sq_fs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-llama-sq-fs', dataset, prefix='llm_expansion_')
llama_sq_zs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-llama-sq-zs', dataset, prefix='llm_expansion_')

# llm expansions with flan-ul2
flan_cot = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-flan-ul2-cot', dataset, prefix='llm_expansion_')
flan_sq_fs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-flan-ul2-sq-fs', dataset, prefix='llm_expansion_')
flan_sq_zs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-flan-ul2-sq-zs', dataset, prefix='llm_expansion_')'''

"# llm expansions with gpt\ngpt_cot = tira.pt.transform_queries('workshop-on-open-web-search/tu-dresden-03/qe-gpt3.5-cot', dataset, prefix='llm_expansion_')\ngpt_sq_fs = tira.pt.transform_queries('workshop-on-open-web-search/tu-dresden-03/qe-gpt3.5-sq-fs', dataset, prefix='llm_expansion_')\ngpt_sq_zs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-gpt3.5-sq-zs', dataset, prefix='llm_expansion_')\n\n# llm expansions with llama\nllama_cot = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-llama-cot', dataset, prefix='llm_expansion_')\nllama_sq_fs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-llama-sq-fs', dataset, prefix='llm_expansion_')\nllama_sq_zs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-llama-sq-zs', dataset, prefix='llm_expansion_')\n\n# llm expansions with flan-ul2\nflan_cot = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-flan-ul2-cot', dataset, prefix='llm_expansion_')\nflan_sq_fs = tira.pt.transform_queries('ir-be

In [21]:
'''tokeniser = pt.autoclass("org.terrier.indexing.tokenisation.Tokeniser").getTokeniser()

def pt_tokenize(text):
    return ' '.join(tokeniser.getTokens(text))

def expand_query(topic):
  ret = ' '.join([topic['query'], topic['query'], topic['query'],  topic['query'],  topic['query'], topic['llm_expansion_query']])

  # apply the tokenization
  return pt_tokenize(ret)

# we wrap this into an pyterrier transformer
# Documentation: https://pyterrier.readthedocs.io/en/latest/apply.html
pt_expand_query = pt.apply.query(expand_query)'''

'tokeniser = pt.autoclass("org.terrier.indexing.tokenisation.Tokeniser").getTokeniser()\n\ndef pt_tokenize(text):\n    return \' \'.join(tokeniser.getTokens(text))\n\ndef expand_query(topic):\n  ret = \' \'.join([topic[\'query\'], topic[\'query\'], topic[\'query\'],  topic[\'query\'],  topic[\'query\'], topic[\'llm_expansion_query\']])\n\n  # apply the tokenization\n  return pt_tokenize(ret)\n\n# we wrap this into an pyterrier transformer\n# Documentation: https://pyterrier.readthedocs.io/en/latest/apply.html\npt_expand_query = pt.apply.query(expand_query)'

In [22]:
'''def pt_tokenize(text):
    tokeniser = pt.autoclass("org.terrier.indexing.tokenisation.Tokeniser").getTokeniser()
    return ' '.join(tokeniser.getTokens(text))

def compute_term_weights(docs, expansion_terms):
    term_weights = {term: 0 for term in expansion_terms}
    for doc in docs:
        for term in expansion_terms:
            term_freq = doc['text'].count(term)
            term_weights[term] += term_freq * doc['score']
    return term_weights

def expand_query_with_weights(topic, bm25, top_k=10, top_terms=5):
    # Erste Abfrage ausführen
    initial_results = bm25.transform(pd.DataFrame([topic]))
    top_docs = initial_results.head(top_k).to_dict('records')
    
    # Extrahiere die Expansionsterms
    expansion_terms = topic['llm_expansion_query'].split()
    
    # Berechne die Termgewichte
    term_weights = compute_term_weights(top_docs, expansion_terms)
    
    # Wähle die Top-Terme basierend auf den Gewichten
    sorted_terms = sorted(term_weights.items(), key=lambda item: item[1], reverse=True)
    top_expansion_terms = [term for term, weight in sorted_terms[:top_terms]]
    
    # Erstelle die erweiterte Abfrage
    expanded_query = ' '.join([topic['query'], topic['query'], topic['query'],  topic['query'],  topic['query'], top_expansion_terms])
    
    # Tokenize the query
    return pt_tokenize(expanded_query)

def expand_query_transformer(bm25, top_k=10, top_terms=5):
    def _transformer(topics):
        topics['query'] = topics.apply(lambda row: expand_query_with_weights(row, bm25, top_k, top_terms))
        return topics
    return pt.apply.query(_transformer)'''

'def pt_tokenize(text):\n    tokeniser = pt.autoclass("org.terrier.indexing.tokenisation.Tokeniser").getTokeniser()\n    return \' \'.join(tokeniser.getTokens(text))\n\ndef compute_term_weights(docs, expansion_terms):\n    term_weights = {term: 0 for term in expansion_terms}\n    for doc in docs:\n        for term in expansion_terms:\n            term_freq = doc[\'text\'].count(term)\n            term_weights[term] += term_freq * doc[\'score\']\n    return term_weights\n\ndef expand_query_with_weights(topic, bm25, top_k=10, top_terms=5):\n    # Erste Abfrage ausführen\n    initial_results = bm25.transform(pd.DataFrame([topic]))\n    top_docs = initial_results.head(top_k).to_dict(\'records\')\n    \n    # Extrahiere die Expansionsterms\n    expansion_terms = topic[\'llm_expansion_query\'].split()\n    \n    # Berechne die Termgewichte\n    term_weights = compute_term_weights(top_docs, expansion_terms)\n    \n    # Wähle die Top-Terme basierend auf den Gewichten\n    sorted_terms = sorte

In [23]:
'''pipeline_gpt_cot = (gpt_cot >> expand_query_transformer(bm25)) >> bm25
pipeline_gpt_sq_fs = (gpt_sq_fs >> expand_query_transformer(bm25)) >> bm25
pipeline_gpt_sq_zs = (gpt_sq_zs >> expand_query_transformer(bm25)) >> bm25

pipeline_llama_cot = (llama_cot >> expand_query_transformer(bm25)) >> bm25
pipeline_llama_sq_fs = (llama_sq_fs >> expand_query_transformer(bm25)) >> bm25
pipeline_llama_sq_zs = (llama_sq_zs >> expand_query_transformer(bm25)) >> bm25

pipeline_flan_cot = (flan_cot >> expand_query_transformer(bm25)) >> bm25
pipeline_flan_sq_fs = (flan_sq_fs >> expand_query_transformer(bm25)) >> bm25
pipeline_flan_sq_zs = (flan_sq_zs >> expand_query_transformer(bm25)) >> bm25'''

'pipeline_gpt_cot = (gpt_cot >> expand_query_transformer(bm25)) >> bm25\npipeline_gpt_sq_fs = (gpt_sq_fs >> expand_query_transformer(bm25)) >> bm25\npipeline_gpt_sq_zs = (gpt_sq_zs >> expand_query_transformer(bm25)) >> bm25\n\npipeline_llama_cot = (llama_cot >> expand_query_transformer(bm25)) >> bm25\npipeline_llama_sq_fs = (llama_sq_fs >> expand_query_transformer(bm25)) >> bm25\npipeline_llama_sq_zs = (llama_sq_zs >> expand_query_transformer(bm25)) >> bm25\n\npipeline_flan_cot = (flan_cot >> expand_query_transformer(bm25)) >> bm25\npipeline_flan_sq_fs = (flan_sq_fs >> expand_query_transformer(bm25)) >> bm25\npipeline_flan_sq_zs = (flan_sq_zs >> expand_query_transformer(bm25)) >> bm25'

In [24]:
'''pipeline_gpt_cot = (gpt_cot >> pt_expand_query) >> bm25
pipeline_gpt_sq_fs = (gpt_sq_fs >> pt_expand_query) >> bm25
pipeline_gpt_sq_zs = (gpt_sq_zs >> pt_expand_query) >> bm25

pipeline_llama_cot = (llama_cot >> pt_expand_query) >> bm25
pipeline_llama_sq_fs = (llama_sq_fs >> pt_expand_query) >> bm25
pipeline_llama_sq_zs = (llama_sq_zs >> pt_expand_query) >> bm25

pipeline_flan_cot = (flan_cot >> pt_expand_query) >> bm25
pipeline_flan_sq_fs = (flan_sq_fs >> pt_expand_query) >> bm25
pipeline_flan_sq_zs = (flan_sq_zs >> pt_expand_query) >> bm25'''


'pipeline_gpt_cot = (gpt_cot >> pt_expand_query) >> bm25\npipeline_gpt_sq_fs = (gpt_sq_fs >> pt_expand_query) >> bm25\npipeline_gpt_sq_zs = (gpt_sq_zs >> pt_expand_query) >> bm25\n\npipeline_llama_cot = (llama_cot >> pt_expand_query) >> bm25\npipeline_llama_sq_fs = (llama_sq_fs >> pt_expand_query) >> bm25\npipeline_llama_sq_zs = (llama_sq_zs >> pt_expand_query) >> bm25\n\npipeline_flan_cot = (flan_cot >> pt_expand_query) >> bm25\npipeline_flan_sq_fs = (flan_sq_fs >> pt_expand_query) >> bm25\npipeline_flan_sq_zs = (flan_sq_zs >> pt_expand_query) >> bm25'

### Bo1 Query Expansion

In [25]:
'''bo1_expansion = bm25 >> pt.rewrite.Bo1QueryExpansion(index)
# build final pipeline for retrieval
bm25_bo1 = bo1_expansion >> bm25'''

'bo1_expansion = bm25 >> pt.rewrite.Bo1QueryExpansion(index)\n# build final pipeline for retrieval\nbm25_bo1 = bo1_expansion >> bm25'

### Evaluation

In [26]:
'''pt.Experiment(
    retr_systems=[bm25, bm25_stopwords, bm25_bo1, bm25_rm3, bm25_kl, pipeline_gpt_cot, pipeline_gpt_sq_fs, pipeline_gpt_sq_zs, pipeline_llama_cot, pipeline_llama_sq_fs, pipeline_llama_sq_zs, pipeline_flan_cot, pipeline_flan_sq_fs, pipeline_flan_sq_zs],
    topics=pt_dataset.get_topics('text'),
    qrels=pt_dataset.get_qrels(),
    names=['BM25', 'BM25_stopwords','BM25_Bo1', 'BM25+RM3', 'BM25+KL', 'BM25+GPT-COT', 'BM25+GPT-SQ-FS', 'BM25+GPT-SQ-ZS', 'BM25+Llama-COT', 'BM25+Llama-SQ-FS', 'BM25+Llama-SQ-ZS', 'BM25+Flan-COT', 'BM25+Flan-SQ-FS', 'BM25+Flan-SQ-ZS'],
    eval_metrics=['recall_1000', 'ndcg_cut_5', 'ndcg_cut.10', 'recip_rank']
)'''

"pt.Experiment(\n    retr_systems=[bm25, bm25_stopwords, bm25_bo1, bm25_rm3, bm25_kl, pipeline_gpt_cot, pipeline_gpt_sq_fs, pipeline_gpt_sq_zs, pipeline_llama_cot, pipeline_llama_sq_fs, pipeline_llama_sq_zs, pipeline_flan_cot, pipeline_flan_sq_fs, pipeline_flan_sq_zs],\n    topics=pt_dataset.get_topics('text'),\n    qrels=pt_dataset.get_qrels(),\n    names=['BM25', 'BM25_stopwords','BM25_Bo1', 'BM25+RM3', 'BM25+KL', 'BM25+GPT-COT', 'BM25+GPT-SQ-FS', 'BM25+GPT-SQ-ZS', 'BM25+Llama-COT', 'BM25+Llama-SQ-FS', 'BM25+Llama-SQ-ZS', 'BM25+Flan-COT', 'BM25+Flan-SQ-FS', 'BM25+Flan-SQ-ZS'],\n    eval_metrics=['recall_1000', 'ndcg_cut_5', 'ndcg_cut.10', 'recip_rank']\n)"

In [27]:
'''pt.Experiment(
    retr_systems=[bm25, bm25_rm3, bm25_kl, pipeline_gpt_cot, pipeline_gpt_sq_fs, pipeline_gpt_sq_zs, pipeline_llama_cot, pipeline_llama_sq_fs, pipeline_llama_sq_zs, pipeline_flan_cot, pipeline_flan_sq_fs, pipeline_flan_sq_zs],
    topics=pt_dataset.get_topics('text'),
    qrels=pt_dataset.get_qrels(),
    names=['BM25', 'BM25+RM3', 'BM25+KL', 'BM25+GPT-COT', 'BM25+GPT-SQ-FS', 'BM25+GPT-SQ-ZS', 'BM25+Llama-COT', 'BM25+Llama-SQ-FS', 'BM25+Llama-SQ-ZS', 'BM25+Flan-COT', 'BM25+Flan-SQ-FS', 'BM25+Flan-SQ-ZS'],
    eval_metrics=['recall_1000', 'ndcg_cut_5', 'ndcg_cut.10', 'recip_rank']
)'''

"pt.Experiment(\n    retr_systems=[bm25, bm25_rm3, bm25_kl, pipeline_gpt_cot, pipeline_gpt_sq_fs, pipeline_gpt_sq_zs, pipeline_llama_cot, pipeline_llama_sq_fs, pipeline_llama_sq_zs, pipeline_flan_cot, pipeline_flan_sq_fs, pipeline_flan_sq_zs],\n    topics=pt_dataset.get_topics('text'),\n    qrels=pt_dataset.get_qrels(),\n    names=['BM25', 'BM25+RM3', 'BM25+KL', 'BM25+GPT-COT', 'BM25+GPT-SQ-FS', 'BM25+GPT-SQ-ZS', 'BM25+Llama-COT', 'BM25+Llama-SQ-FS', 'BM25+Llama-SQ-ZS', 'BM25+Flan-COT', 'BM25+Flan-SQ-FS', 'BM25+Flan-SQ-ZS'],\n    eval_metrics=['recall_1000', 'ndcg_cut_5', 'ndcg_cut.10', 'recip_rank']\n)"

In [28]:
'''p1 =   pipeline_gpt_sq_zs >> bm25_stopwords
p2 = pipeline_llama_sq_fs >> bm25_stopwords
p3 = bm25_bo1 >> bm25_stopwords
p4 = p1 >> bm25
p5 = p2 >> bm25
p6 = p3 >> bm25

p7 = bm25_stopwords >> pipeline_gpt_cot
p8 = p7 >> bm25_stopwords
p9 = p4 >> bm25_stopwords

pt.Experiment(
    retr_systems=[p1, p2, p3, p4, p5, p6, p7, p8, p9],
    topics=pt_dataset.get_topics('text'),
    qrels=pt_dataset.get_qrels(),
    names=['Prototype1', 'Prototype2', 'Prototype3', 'Prototype4', 'Prototype5', 'Prototype6', 'Prototype7', 'Prototype8', 'Prototype9'],
    eval_metrics=['recall_1000', 'ndcg_cut_5', 'ndcg_cut.10', 'recip_rank']
)'''

"p1 =   pipeline_gpt_sq_zs >> bm25_stopwords\np2 = pipeline_llama_sq_fs >> bm25_stopwords\np3 = bm25_bo1 >> bm25_stopwords\np4 = p1 >> bm25\np5 = p2 >> bm25\np6 = p3 >> bm25\n\np7 = bm25_stopwords >> pipeline_gpt_cot\np8 = p7 >> bm25_stopwords\np9 = p4 >> bm25_stopwords\n\npt.Experiment(\n    retr_systems=[p1, p2, p3, p4, p5, p6, p7, p8, p9],\n    topics=pt_dataset.get_topics('text'),\n    qrels=pt_dataset.get_qrels(),\n    names=['Prototype1', 'Prototype2', 'Prototype3', 'Prototype4', 'Prototype5', 'Prototype6', 'Prototype7', 'Prototype8', 'Prototype9'],\n    eval_metrics=['recall_1000', 'ndcg_cut_5', 'ndcg_cut.10', 'recip_rank']\n)"